In [ ]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *


training = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "sentimentdataset")

validation = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "validation")
testing = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "testing")

#remove nonASCIICharacters
training["Text"] = training.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)
validation["Text"] = validation.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)
testing["Text"] = testing.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)






In [ ]:
def to_class(s):
    match s:
        case "Positive":
            return 2
        case "Negative":
            return 1
        case "Neutral":
            return 0
    

In [ ]:
training["SimpleSent"] = training.apply(lambda x: to_class(x["SimpleSent"]), axis=1)
testing["SimpleSent"] = testing.apply(lambda x: to_class(x["SimpleSent"]), axis=1)
validation["SimpleSent"] = validation.apply(lambda x: to_class(x["SimpleSent"]), axis=1)

In [ ]:
toDrop = ["Timestamp", "User", "Platform", "Retweets", "Likes", "Country", "Year", "Month", "Day", "Hour"]

training = training.drop(toDrop, axis=1)
testing = testing.drop(toDrop, axis=1)
validation = validation.drop(toDrop, axis=1)


In [ ]:
import re
def split_words(xs):
    empties = []
    empty = "" 
    for i in xs:
        words = re.split(r"([A-Z])", i)
        words = words[1::]
        for i in range(0, len(words) -1, 2):
            empty += words[i] + words[i + 1]
            empty += " "
        empties.append(empty.rstrip())
    return empties

In [ ]:
from textblob import *
#process Hashtages
training["proc_hash"] = training.apply(lambda x: x["Hashtags"][1::], axis=1)
training["proc_hash"] = training.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
training["proc_hash"] = training.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
training["Hashtags"] = training["proc_hash"]
training = training.drop("proc_hash", axis=1)

testing["proc_hash"] = testing.apply(lambda x: x["Hashtags"][1::], axis=1)
testing["proc_hash"] = testing.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
testing["proc_hash"] = testing.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
testing["Hashtags"] = testing["proc_hash"]
testing = testing.drop("proc_hash", axis=1)

validation["proc_hash"] = validation.apply(lambda x: x["Hashtags"][1::], axis=1)
validation["proc_hash"] = validation.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
validation["proc_hash"] = validation.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
validation["Hashtags"] = validation["proc_hash"]
validation = validation.drop("proc_hash", axis=1)

validation






In [ ]:


training["text_tb_pol"] = training.apply(lambda x: TextBlob(x["Text"]).polarity, axis=1)
training["text_tb_sub"] = training.apply(lambda x: TextBlob(x["Text"]).subjectivity, axis=1)

training["hash_tb_pol"] = training.apply(lambda x: TextBlob(x["Hashtags"]).polarity, axis=1)
training["hash_tb_sub"] = training.apply(lambda x: TextBlob(x["Hashtags"]).subjectivity, axis=1)




In [ ]:
analyzer = SentimentIntensityAnalyzer()

training["text_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["Text"]), axis=1)
training["text_vader_neg"] = training.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
training["text_vader_neu"] = training.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
training["text_vader_pos"] = training.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
training = training.drop(["text_vader_scores"], axis=1)

training["hash_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["Hashtags"]), axis=1)
training["hash_vader_neg"] = training.apply(lambda x: x["hash_vader_scores"]["neg"], axis=1)
training["hash_vader_neu"] = training.apply(lambda x: x["hash_vader_scores"]["neu"], axis=1)
training["hash_vader_pos"] = training.apply(lambda x: x["hash_vader_scores"]["pos"], axis=1)
training = training.drop(["hash_vader_scores"], axis=1)







In [ ]:
training

In [ ]:
'''
    Begin logistic regression
'''
'''
solver          penalty
lbfgs           l2
newton-cg       l1, l2
sag             l2
saga            elasticnet, l1, l2


'''

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

classes = training["SimpleSent"].to_numpy()
#features = training[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()

'''text only'''
features = training[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()

#validate_features = testing[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()


#for i in range(100):

averages = []

total = 0
length = 0

#lbgfs
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
    lbgfs.fit(x_train, y_train)
    results = lbgfs.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

''''''
total = 0
length = 0
#newton-cg
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    newton_cg = LogisticRegression(penalty="l2", solver="newton-cg")
    newton_cg.fit(x_train, y_train)
    results = newton_cg.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

total = 0
length = 0
#sag
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    sag = LogisticRegression(penalty="l2", solver="sag", random_state=i)
    sag.fit(x_train, y_train)
    results = sag.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

total = 0
length = 0
#saga
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    saga = LogisticRegression(penalty="l2", solver="saga", random_state=i)
    saga.fit(x_train, y_train)
    results = saga.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)


averages





In [ ]:
from sklearn.metrics import confusion_matrix
matrices = []
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
    lbgfs.fit(x_train, y_train)
    results = lbgfs.predict(x_test)
    actual = y_test
    matrices.append(confusion_matrix(y_test, results))

mean_matrix = np.mean(np.array(matrices), axis=0)
mean_matrix

In [ ]:
import pickle

x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=0)
lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
lbgfs.fit(x_train, y_train)
with open("TWEETS_LOG.pkl", "wb") as file:
    pickle.dump(lbgfs, file)
#results = lbgfs.predict(x_test)
#actual = y_test
#matrices.append(confusion_matrix(y_test, results))
